In [4]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
#总计20640个样本，每个样本8个属性表示，以及房价作为target，所有属性值均为number
#目标变量：平均房屋价值
#输入变量（特征）：平均收入、住房平均年龄、平均房间、平均卧室、人口、平均占用、纬度和经度

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

input_shape = X_train.shape[1:]

In [9]:
print("X_train:",X_train.shape)
print("y_train:",y_train.shape)
print("X_test:",X_test.shape)
print("y_test:",y_test.shape)
print("X_valid:",X_valid.shape)
print("y_valid",y_valid.shape)

X_train: (11610, 8)
y_train: (11610, 1)
X_test: (5160, 8)
y_test: (5160, 1)
X_valid: (3870, 8)
y_valid (3870, 1)


In [6]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# 创建一个执行层规范的自定义层

## a.Exercise: The build() method should define 
* two trainable weights α and β
* both of shape input_shape[-1:] and data type tf.float32. 
* α should be initialized with 1s, and β with 0s.

## b.Exercise: The call() method should compute the mean μ and standard deviation σ of each instance's features. 
* you can use tf.nn.moments(inputs, axes=-1, keepdims=True), which returns the mean μ and the variance σ2 of all instances (compute the square root of the variance to get the standard deviation). 
* Then the function should compute and return α⊗(X - μ)/(σ + ε) + β, where ⊗ represents itemwise multiplication (*) and ε is a smoothing term (small constant to avoid division by zero, e.g., 0.001).

### tf.nn.moments(X, axes=-1, keepdims=True)：计算输入X中每个特征的均值和误差

In [7]:
class LayerNormalization(keras.layers.Layer):
    def __init__(self, eps=0.001, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps#平滑项：一个很小的数，防止除以0

    def build(self, batch_input_shape):
        self.alpha = self.add_weight(
            name="alpha", shape=batch_input_shape[-1:],
            initializer="ones")
        self.beta = self.add_weight(
            name="beta", shape=batch_input_shape[-1:],
            initializer="zeros")
        #在最后：调用父类build（）方法，告诉keras这一层被构建了（设置self.build=True）
        super().build(batch_input_shape) 

    def call(self, X):
        mean, variance = tf.nn.moments(X, axes=-1, keepdims=True)
        return self.alpha * (X - mean) / (tf.sqrt(variance + self.eps)) + self.beta
        # * 表示逐项相乘

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

#保存自定义的参数
    def get_config(self):
       base_config = super().get_config()
       return {**base_config, "eps": self.eps}

## c.Exercise: Ensure that your custom layer produces the same (or very nearly the same) output as the keras.layers.LayerNormalization layer.

In [8]:
X = X_train.astype(np.float32)
custom_layer_norm = LayerNormalization()#自定义层输出
keras_layer_norm = keras.layers.LayerNormalization()

#计算两者（两个向量）输出的平均绝对误差MAE的均值
tf.reduce_mean(keras.losses.mean_absolute_error(
    keras_layer_norm(X), custom_layer_norm(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=4.1984418e-08>

In [ ]:
random_alpha = np.random.rand(X.shape[-1])#随机生成8个α和β值
random_beta = np.random.rand(X.shape[-1])
custom_layer_norm.set_weights([random_alpha,random_beta])
keras_layer_norm.set_weights([random_alpha,random_beta])

tf.reduce_mean(keras.losses.mean_absolute_error(
    keras_layer_norm(X), custom_layer_norm(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=1.8310335e-08>